# Gleason Pattern Spotter
The Gleason Pattern Spotter is a web application that takes a prostate cancer image, divides the image into a grid of several boxes, and then detects the Gleason Pattern for each box. 

<font size=3 color=red>!!! Please be aware, that the detections are far from perfect and should not be trusted !!!</font> 

This application is the result of my approach to the [PANDA Challange](https://www.kaggle.com/c/prostate-cancer-grade-assessment). Due to bugs in my code I wasn't able to submit the approach yet. This application is designed to inspect the image, the boxes and the detected patterns.


By pressing one of the buttons "Karolinska Biopsy" or "Radboud Biopsy", a random image of the one those Data Providers will be taken, processed and displayed below. (Additionally you can upload your own Biopsy by using the "Upload Biopsy" Button).

The image and its predictions, will apear below and can be interactively analyzed using the toolbar in the top right corner. 

| Icon | Name | Description |
| --- | --- | --- |
| ![](https://docs.bokeh.org/en/latest/_images/Pan.png) | Pan | The pan tool allows the user to pan the plot by left-dragging a mouse or dragging a finger across the plot region. |
| ![](https://docs.bokeh.org/en/latest/_images/BoxZoom.png) | BoxZoom| The box zoom tool allows the user to define a rectangular region to zoom the plot bounds to. This is done by left-dragging a mouse, or dragging a finger across the plot area. |
| ![](https://docs.bokeh.org/en/latest/_images/WheelZoom.png) | WheelZoom | The wheel zoom tool will zoom the plot in and out, centered on the current mouse location. It will respect any min and max values and ranges, preventing zooming in and out beyond these values. |
| ![](https://docs.bokeh.org/en/latest/_images/Save.png) | Save | The save tool pops up a modal dialog that allows the user to save a PNG image of the plot. |
| ![](https://docs.bokeh.org/en/latest/_images/Reset.png) | Reset | The reset tool will restore the plot ranges to their original values. |

In [31]:
import numpy as np
import pandas as pd 
import io 
import random
import itertools
import numpy as np
import cv2
import time
import torchvision.transforms as T
import gc
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show
from bokeh.models import Text, Rect
from fastai.vision import *
from PIL import Image as PILImage
from ipywidgets import Label, Button, FileUpload, Output, VBox, AppLayout, Layout

In [32]:
learn = load_learner('', 'trained_model_fastai_resnet50.pkl')

In [33]:
IMAGE_DIR = '/kaggle/input/prostate-cancer-grade-assessment/train_images/'
def getSampleImage(provider='radboud'):
    """Given a 'provider' and height 'h' this function will return an sample/random image
    from the training data."""
    query = train.data_provider==provider
    filename = train[query].image_id.sample().values[0] + '.tiff'
    info = str(train[query].sample().to_dict(orient='list'))
    image = openslide.OpenSlide(os.path.join(IMAGE_DIR, filename))
    
    too_big = True
    i = 0
    while too_big:
        w, h = image.level_dimensions[i]
        if w*h<2**26:
            too_big = False
        else:
            i = i + 1
            
    return image.read_region((0,0), i, image.level_dimensions[i]), info

In [34]:
provider_urls = {'radboud': ['https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..vVIFoWl8Oz6VDmPYgDdGyg.GhuDcULwXzqEGpYgQqWpt4GU2v8UmWFr9qMLO8BICl9pNWferXO9ywvFmb-mlJooAGiosw5Tf5NmuESv8NvkBZ_95xicXJCpt6r1lpN5VvVhmysogggmFu3xfdessjmsQ22woX7VhZQxw-bi3Yl5cO-GRtihymyXoLFCbXIrUhwWArUdvtlx5vFI7pYpr9cRWNM3NXh4wrYG9xFzFDk4IjgW8mol6R_dNLZKmz7KBwAVoE78J_12qtgGE71npaBEG33qWvaU6XqQAnrtjShzsA1C1mCDFBDdu9-yfo5-T9FoeK6GesdFDUThQ0-eKkqsOayileIsiewRcupTxvPZLmKvDgARXJzFMVLBOvRMk6Val-TJ6w_20jWrncmmsk96UsyzE6v6pif5YzRaxVI4uY_BD7bcYt6bKVrBHG_3F491x89E6TdwgxKP73sJTb-kjw97Fdpjpt4IQnM5xmcdvDNq22aykIg1BOhE_ycqzpTYYMtX07TnICpdqGneE0D3I6N9R3q9RvFyQ1p6N6R3X3fTsDhQ1MYiUo-X8cRjhOa8hClyd5ivLhTK6C0Bg1mMNJbsRnAmr5izfkiEkq5kpAyvv6bV0YCf38FdSuSzGr6XGzWy522PMOwHvPgDPbyTOSjd5B_XgQ9ZxP4on4XiqQ.BV24i6TywB8cIBX2AWXaaQ/658be675223ffa5509203009eb7bba89.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..2dxnG47ZsZy6wGd6KrHOPg.eubnYLU2QhD9eYrcajAwKpuOR4Yy1TwQLXNV_GRYFEdvCkSYRjJON9AquoW2muPIfwy2bXykYKzI27tZXLcjlw7eCQdnUxrJfdMQE71SWWzPo9ypnGsHi5bdTSr8AZnsA78MZQkZpJ3wtxUlimI6pZRuJ_U4Z5xEWMqRLIic3fBOsznt5sYAyX1EIRvYo70lcX3Id-AX15phNQ94QfFUDRZUExO7ZLzITKesAeHIkWMhHN1rYgIaaTFFSAvm9G1g8qh5lQrdK0BiAxKsjHk3KJ5eb0jiLwK_glsK1IUtIdcI897G9aOFk7eoaTEz84p71pMDDcDDsW-l_A2fiYjJC61qMgczpe6aNpWkc0QQfT91YXnkOvFOCxdtmnEgAHSEA-MpygVlqcs12uJzFt_M6raqeme5ab65wVy898FuZ9WYYq3ZSTA9_zFF5UNu1CQMfHcQcUcYtosHo873TQVQEpRVUr6DMwQacRHdHJj82ZA4Im5V7DmhMMhXWW9YpUZaFx8DVN3_jykO0RixwyWBQZGI5u1pI00usSuscI8nZ-1J4Cov_KV3hfujxrs4MY8aGpavOaim37zeeVG-4BJPJ_KNGfeDiHriJ9Xac3MKmOIFvy2q3fSNYx5n-iXCUV41WvrjwncsuOmSgdIWV3M8qg.952tgDPk48kqff4xppEV7Q/c306e9bf52ed33801003d17ae3f60d32.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..M4Ykrl1q-jk_OASYRuN9XA.uNYEojC4sQ_W77QXnDqKzuODzWjfk6mFhbyur-MZQph8daTqREklrTw8pZgKvcTUfr9YNzFMDk1AMYkrTtoPR-Uuu_rTKkBJgypNRDuEegNsZMzfTemGxL4RcfV1qHkybvTo0PVd60ZdrspJ6Y8q3sA9LD91eOHN2lEozX7wUQxQe8RSVWZm4hO4gQHbJnHwRuznOtaTHuVsJnpx_n8NQPBLT403JqXFtAbdJje7Rz72AB-4qldIdFgYJDXOP6eP4I9YeVh-w909echGvtgCS53rU-QyOucMivhJVKaIOqbV-_LgAvXWZ1J43T6-00Py6FP0kQcE6QA66YIKtsWX2yba0GQ1qEW_jrntFwtyxV0NHOLpC1ZZJDF36KdpWIHTX_J0OXFDdqvjmcdxBxX_lJxbKKdqbaYl-gBxuCe4AnL3cNejclNVeZuFbzmcun3wHkAiWeMwEy1e2WyfuRAvgKOzGTvSeDwRtuqb_Pcx0Z3ittHLVPheOpX6LMYG3L7hkGQIm3i9ovTlDt2Tj0EVrv_RXiSkwzt5DRJFkfHBrKvNiXs6lY65hZk3evIesZR2dSA1oEkpOq2rpJ4xfFCPhkvLLMlOg90Aq-tT3GjMMxCxHMH5FuJJ6KJX1G6gxWd0zOBZbVGy53vDQjxdVPQPaA.sszqsGJY0291XLjXN7fZaw/b0ee38976e71dfbfba705974713ee612.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..Rv3mT9jv3IaMXfvvqprN-w.u5xa26zaTyaFAs2KfKVrF1twAGBRG5IAepNWSGudiJoE_eF0Vf9Rpvs9KWFn0UM1tAE4JfANu_poOWj8v24ldPunYv5SIF6GSI7UqSs5tY-3ip3emL5JLsvciOu9VWMQRlxGKqoZlk3uZVjgDOJUp7vRm-fupBEVvoulItt5XlMYALy_RuwAqO3nolxyql-2DW9vOl-7ZLrbC0-guE1uAlg7JIZrSe_TFgXPpEZnsU0kOZSuhCtQ3kISs7XofB6Enlpyj-BfL3BW9uCak8dGrPt8BnWqQ42ikovEVhhczuaAttZEuVTUL_TmOdInJR8BXJObv0YiN7lro2faud8yYWWEbjdJjaGJZHtKfIBTiMHxgDSFWaXiZLe6zzGaCcSUY2mh8c3PaQDY94j-i5ARdfoJCCWM3MxrAV93XbAcGuCEkl8j13Lotl_dl_Unk7oqYl23TGJMpy_2-6etioqUp02l_O0M7UDMDKZ9BJlN2rRsK4E2Hgye6hQa5AWUUoMmJoszxYq9nwPobgOxAnnNxper0owRX2gq9QxWun6D4oKAejxxjR4gVpT4W-UabpR-DbKBiCxpiPX5m5zVvwXLep9XsxKRfpa2xWQD70lkq4mRA9UUr2s38DVuttT5crC_qX-dH81FMvLbDAkIZCAvVQ.qjZFB9giX11jA25538QA9Q/8a21fc0aed2fd06a8b3fe4824c53413e.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..DC-NSxM8H4T6R4a4aQ_7Kg.AAGAlPlYc4nEdx2kkJ2bw6u0jUNOtLbL35C7B80dOSJhgSOLt8uAhx24kkn3SR0zA0SlGVpF1v9wze4tI75lBv13D1eJ7yQeM3bIq23c7-scFhqS62AAggtiWuIHWT5FGcqiY4rN2Jmwqp9dyV-UjwU8VcvI4nlbJkPIPUlEwRlNE50LqEjgfI5PZtbxD_5gBlalhI-AQF5L0bfFnPEdiEPaZCq593BdeHsBHdeO0ClDsG3j0FP41P2RahyHLIdOBjG_LhgH0VO70i7fBuODiAgwvb8yDbjoQzkrodHrbb7IzlEaJgkz_6UU3U0Dr3WDWf_cZINXYhWguxjZl7ImbNFIaVDTfQ6wzCap2MRfyMfXkzLNrJD0x0QQIvdjXX5v5KGkFh3eVcFuyaURLQeLrxGQDBeytXWfGOFJIRYGhM6ljsCvg8ntxLWcsWMATWi1qeSnSOsYylgDi9EJce0O99uRJt4y0dDjDpHHA_YwbiJO5a9Jg52J1qepB6uRBwybFq1szK0lxAia-gNnTpUd9sM8G9KgVxHmh-5mW-eTZKOtDDYNjWn2LQmTQd0rQ9fEbAfWpgyHcmPKnCLqooqR-FEm5_MIrb7r3rGWzzD4YikYfT-AvBmk0K4rHgaUHA7RThM-39COmAeifraXCaXwrw.TNfO0tZeY2I-Oy6PHBRN2Q/6a5cc0b66b8d4af9c13f1303c6de5ffb.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..07ZOm7eN67ROXowEV58pDg.2a1T1TwkZayyumyFstsi7SFbxK7LdwOuWMCOE7RQjwTQF0LNW2-bvwMv6EZQavdhm8DvxFVKSrnz9rXVbCG5DAFscn5Vq2Gt4j9dVtZPPfz--0P6gMzJ8muwHkjMEQtSIeYy6ZmwkMgkJxJVlBrpySopdtfgzLc61Hyxi80_o2qA1ioNq9etIIaEBOs4dcdC-BrX882brUqEEvrSEIti36rQsBGOb58j8k179PjkK6pSDIs7zbSnblJh_kpc2JYBd4lSilm8NVRA1GtqBHDIeDewlVGb5CC6ZipHLbqg8-Sxtj_V1BUSJySO-Iv9BVujN3MK2VRTk3zdItZTCHh7ELUrVA2W0ImBOHCv2W_e1grEkrUejxRz2ol3mFW-P1ZU4JypC3_InyyRs6kMq8yox8UnCkowDJb3YjUgvvOTi3tQfFExWdqvtG5ZNGsXGupKUEw4UemIdAZtAeXfVrXhwNBwWpvXS_B9HKH_uDjRI7GfqBSvkYYOZOoT_w2acc78i9nd0YyBjarBjNylKdIWjZ3PAzSns-q3M-J6pGqrtQAh1pXWuWXLBwHd3e7SCgwD84BbkYcD0SNbihqHoyhzv5K7GZBdxCLoBg_hynOy1KzQl8QPqcenPPmtaLyqxUTFUJg-_MiYIP_CqLrHhS3nKw.tOMa3bZm0GGiMoA_iAJy2w/58d13c5b7c1d86814e7a9451fef0d9d5.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..rbtNT_rXWhHOkRlBvQwAuA.Z03k8tL_X0d3xShwuwPlwH42hIReurW3aMrHzrJCW8AQ4QQ1tnPA3dqImPgFYkgpnLq6SZmZ6IzFgMHViw1EFZi8LxesTEWD6KtpqujEjzfRTWbaWFXhe5aGVvtWChE98JtUA7xHCQT_PAY6oiTzjuAhqwwXQ4A5mhQbdj7ScpcdANZBAVO-fHDQk9gEMxmCwgWJZu--8nko6FPD2FBSBFe1eGVmdKzrFJDOpWmwrA30EGPVpYSWNUbkMWZ1rD4pD68IPLUL210ieN9EUjMcAXd-WVYyfrSNjp2GRgcLL_jhJGWzGgbiuKntIO2V6tlg4TLqFGivg9Yb2VVKKEuBHbDDt--pvYAgHURuNyUCkHhymRuKkA21yttLrkoYFzXMyU0k8dNHAMq8PwzoW6FUpkKBSMDrjLmh--IrvSHMj9z97NiBWNC1jAqzsF0GYxV4cDDNCv6RgoqEgrnHzXGIl8ekYRG2S15181jNtwbfqEpIuXxsHfYoGmcKCiK5w0p_9fCAnteAZVXHt287SKyIW5Uk3UBsLb8PQQpQmTzthTlAa1LjDTE6AtCmcI65bxCdBVpTDTVEjO8TqntL3a3kiUHqGFJsJs2rBTk1YFWc9b5tW__zzTWbasEbsm_RJVua5uUgM1fs5uKG_MtCM-GZrg.JuqGdLMddQwfU-ltgmM2Uw/36eeee47a868b8ac66b6ece9e6372984.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..LMunPedQ-D0xam3GYMUZag.17ZlGyBrL-dyayAjdohDWTdVwZIorT6xej27wb67st2gFQWgE851bHkKRAMW7H7R2t3tx3MRuYiZhws913rsBUVs9T2XHX8cMBeNgfqXrXN0uv_DiRiULizCJ7xB8OdqA8rxuFXO7o2SJ5Dq5UUypXNgPbHwgbogafBUVC9IFGvMkEmNc0EYkYI39aXReH-bdclJ5kCd-Al0Y4m3XTNxFcykZaMIuKgl7idDabwszrBOKfF2YRRomsKN7bT-qFkoJkBtzjGJ6umNuJ8--prrrCcFVzlGPRzJ3zWLD_v_tko2avEd8TJqqBxxMOst9vhV7R4QVXw925I48h0za1DoDmCvl4oaZWCdun6FCpXWsdvrvbwz5BYTA3ptuepACfDa3JwDvaRX9xGwZCYKo0G3QVy2TSLEk36jdd1-RSmr4cb5mzfk0gqsYuAyfzXyl3f8pU8Q3X8opd1PlXF3SqG3u7gvRb9zFOULVmaaWAiLLv6LroKaf5O-sFlKHbaaWTtsT34k1XU0yXqYSJ50X2rXoSwsRsEZQMiyKolwZn4-ZQ9zV22QQDImQIOYG8LiQanswQV35hlRwmDdT_Aepyl5eoLk8tIO6IwrfWLb5Cmv9vIE9xZLO56vblpLfBU4fDYNxmGDeX_k_ZB5MpYkI16KZA.DaE4AF3Ll1mdhvGoe5Q0ig/35f629e359399d425289242b698b3f66.png',
                              'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..6Z3Dm6hGEnqEq-uUNfxvdw.i8cBbvoiGtLf0Bg-MKY2fl09nTA5LseWLZjqOFt3kuKGWTXxDIDbVCvVJGZmUUeqPZisJ_U06K1EYNwNgtapqtJi0OAkGjhDsQ8NlFLm7sRn8Mw0o9UoKgYzCMfgmE3C2aIfGJ3AfujPCgIWPJ-K3oiii3_ImED56FNKceO24AVIm-V-80C0v6l-wrZ3cQZUQQjTZhRFnSZeVO3w-Xsyt3c3Y0Tll5jbhgskvJ-FtM4xhwrDV23JnqZd0KNaoY1ScCXwXCb0V-XmJKZNEmMQpvz2okNjD2cbUtaRYuxHyXS3JPyIATIFsm8Kfb2lpB6SurZGDqgRBd64gmGjFKP9WHFYBHhP90PxK4fMkfCG3RWkRHls5Yfzcm3hLIGiHmz6VE9telOI65hPMUR283FMAchWe-L5vsrEuS4ImBxMDl2PO2Y84JkEXdT07u6YixhwVnV3_dhnD77H9Db-k9hsxr5A_Q79oWn2FP44L0cL6CATNQ_4r1YaWgeyw10DPwEN25ps3JWtjAmwcIHEWqOpQaO-bVHXuRE6XQILbWXmmijiIvHa7ZUtDOjV6Nx60i28RVj6wfVAkaQ0qDzUKD7K0k2X1ufdJ-XgiAYK592OGcF4qT6bQMPL4nAnTN6UkbkldU8Gb1cN17BCSnDAKrSjEw.G4P6SiUhz7bECe1oqJRdYg/345f3e86a3dcc26dce73ceeacc5a4a0a.png'],
                 'karolinska': ['https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..epwGXd43qS7us3D2vf2qWQ._ESHk-1hSsAK1BNSn-UmyavF0AQUKjLYzBhBKh9HcPVK-NRRccKKXiM1-1MCjXgp5Gnnueaxky1wZibn4W3QYuFQOAFn6A9Z7xsI-u855tjNzTJb1tUmJgtMboDrLlysKrMIos5R0DpxT9A4DBfuORzcSEcklnd4MaE14TorB8kZApR4vzjHi2nF8wkwG8__Doo86-oCNco-_kFiaAi-SsAiuKB3cnqJg9AHM0LZDCJBdpKOxQ3wsb1-yDbLg8Tb8TD-vhhPupkRj7T0GM8EwiEQBfPRFGxjduLN8myM1QzCGsC-daB2SMPz8ydB7BYV4hac2mTXuqlrhqhVZuzPMBJR5E0TO5QeMeKnigRf86u3v62lG2TwRftv8UNWbVWxnF0Tj7gOX0da1p_S7p5LyF47CY46Eg13e-l-y090NQgX5DPQlmRUKEXJ7guMsSBjTOYs-NLyIxeBXaAkVWFxIBREqceXCd6VOUEWnuZ9023TNI5M8tyXfYEX8e7j6PrrxTXHQrsK26AgBpQYfykdCZ75SP9jBnLBm4CUtnu14_Gu0BCRkJnSeeut_KBnZvp34eOIYVxAZOKczoAEUxVAk_JdL5BNzkOOUXQZM-Rr8sbBbYxjWwpsvcVOv7MtKgBd8j-LK-w0Zll5KURmEUMwQg.Mm7sZzwrBXLAFF7hyjQFGw/c4b7ca39cc464a7af7f80fb9f4bc05c3.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..TVztHfrSdd9W-TCOZXzMBQ.kkJQWsR8k1rGWhJdaewdpu5Wl3s4ZsFbCjxOiau0z_V1mCNHe44CaMTbFIyvjdv9_duvltF-Zdu_cNNWd05FllJl2DJ8hDzz0EonByDKTeLI8HVhVKIhF1vBCghFTJmjkcmY6RJigWuZoNvFsBR893oI6XkxEGwBUKf0t8CfJmM0wcRi--pKs00zCT_DXa_8zOyBBEsgYuiLmI8PA5ebpibqeDJ272MKIs3Vy_VKeedTE-ZsmxIGmK_NYILUI73ISM9nF_Ar0CCjru005kJV9ZKKWEuHxj-JwSUfTLRHNQroo018yeTgRn-6B5lZmRz46lMs2jI8xaVQY_QtsLVUUEgqAeKcra1aooBNt3w9jPuvRyIXv-odpCXUl-7cpGVhz2te1lVmDCWmedgFU5ayz6bNnkSKzM9liQRruAwQkXr_KkBFgNCoVX0BZ5gW5czdcO5oZbzLJECAmLPoG1wVe4NAQGBWP8hbhCPtzxU0zmAUkNEibpEed4fRvGb3P5SzjcjWRv9lUaJFQWHNW6U-kgmKQiF4v9oV3dd6IRFSQn1tbRKraZswncwcpbd8Nd3u4mMFXzF1-ZBN4Kyy3v4Q5xPZJNwj590yFQ5pQg043Sc8tMMYUF8XDVArbz5cN0RiBVvLxjZjFuopBJzNtyF0_A.B4pNUzcQIr_nZNaVVpp3Tg/acc79fc48cdb1a2c730a0b2c15b0355d.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..SWRVAUhkApG0xdq4auwjag.drDtw2Q05cEsJZGbwLLyCvndyaK416a3ek42yjkyIZkdBVSw9IrbIAxpO_djZUGS67S7-Jnw7gWExD5YR1UIFR9inuQMqiAE_6M72szx4GANjfwrBR-wBykSTWPCAOYiAfLuhICXDRs4KkE0I6O6L1yL3q--UUt4GWbWYQWzgq1IoHXah43dWPGmPd44vAZCSDQJ_3Y-1ZBdRCIYFZzr85PNbKZYLG_3QZVeoeeCldWYD9ACXRFxsAaJNsKOg_Cgzz653KeHCUdrMCAGFL2tAX-UHJRD0STw233frnS6NrvTaprn4tauMA2JgvZP4XaCY_LW1nBR-nKp2ec_YPk3v-7Ua3kRuV4vScbkFQRdaHKthvwORsdvrU_rHkhd51q5Wf2ipMlXGUIqFnPwLAL3sV7hLfD1a7KAuwGpRRUGGkandw0XWVLc60R3degaC4yY8pCo6Thy8c7GaHiL-84SwVlTlNv0nASz_7hOwO1OpWLwffYgzz22VjoFdVg8jnZukpWrNn5SXI7wkmouxv_YyBVuRdjpm7kS4IBPtc06KrrGsfyAGV-VKRe4cMf2mBBUsSePGuVXPePoen6qn5IKDp2_vMljFVaULKHzxCHDyg6dq2MUGoH1qiDXmrA_g2WfkMkL05KkcP_OmkGDGHlPzA.FhpMfZdazJdX0eRsV6qtFQ/a6a07e971c280f5cee808a92a5b06e70.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..mCr6LO4AL6p0HXVbE-5ixw.EqlUF6yZ5Z-btKUlMMWSHUqsPqfOXERg15Mk22OgR_wDhS1BtNyF-u2wnKoJgOqHdgzzB3iETyZDEejp0pHXN6XszneeFdc3faQFaQOtQi2ki-2WuTiIOJ6g9ut3MEKk8wwd6L2vVcOR8VxgVYIPgM6sJHYfJzp-Kk8qzcsj-7izSiIupL6Gcdq__psprSx6bzopUslZ9lU8pE7Eo0Dvj1do5YSVFSUyEeVe5dqjac5vxNka10rBPBQRTKJMKtrFSyhPTMRz2O1rdTqdAzYyKif3Qqu4Y8fMTURP6XJSoCnQUVYvmHKS1HRz40UBnp6kBPPB_ReQj_LysehyOOc4Qlt4cJJClJuBW0V-IsZlwXzSXKWFDfxelDuJn-CLzur2t5GFOGqfV9lkW7QS42xNxgrP5NJKUzppWV-mzsmd3W6SXx4NUHsayF-vuFzwLIZi3dVVKU-M7j6Gz-ale5BMVaghE5fBbBBJU1d3NGodXEArjgtOX4WjW7lTFkQj1Pe5OPkHqOxWCJ9CptAVkQhnVIuFYsdfMsHqdyI0Kfgvi6oBHsWGFGlqV1XD7nTy-mACFUgary4esjYWPQMFNeXtLHcCkhGghrycscBEAnuv9GMvXJmh7SIWwZrMXo5vnR9nMJZ8WuhcuXpCGHBjOOVMNQ.i3NoIdXgfiHfXiekm_EgnQ/89d137c7fb29df7596e6824181b7d5c9.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..VEeNgQetd1v-DP04WT4emg.6CaMibMXGuWX1r4wkxj3qdPH1Aw9UJtjOJdQgO1le1195C05cWHg7vVR_G9BqcGL6Lk2MrM-k-oFwUxtzIotGXANhdjjzg0DhXcrctiS4nwpFrNa3Ewy_WJ_nbQXYCIXBhztMnSiSkx2tC67UVSb_YVkg-NZ60vNri1pJjlAi1zB7tT6Mgvz4nExoD_0haGPtcYXbzG-g4BSVQHKOkJ_wUA1NXW8nHOXV0bRBkv4p8Er6J-CQmDkdx55ldYtUx03E6qLFk5pj2jDAeFbgAjh5vbPdTLGWQWeHHgfWc4M8_cf7xGxyiO-X_--DNnnbXtRWyOK-ENPX7HKfCkc995Es57xEYS9fb0m3pwfEg3zU1dIxLkgXUJDgDRg61on-4YEcTZdfTrCKbLOu-PiXrVCk2UXl4twiyWHTxSIMvDcFM5VRjgDcpoeZi9JrAbZcGYQ0ame0vPxDrZd8lkG_4myibsD77oSRSJDki5B3T_nH4ODFJlcXbsrEYuqi3Lto7-4T3pKjFsf3er5l4si0pSnz27tkR4yDM-RwdzYVBLvkHwsr7CpHBimZHHvb7laSw1eNnLP2jdhVR-pmkek700kU8y6SHc6LbSVIScvbozVYP_Gz1QympzlXJ7ac5-7JvsGjLG3y23aqNS-QpkWsuYQsw.frsNDmD7L67OIw8E6sd_PQ/fe1d0b156ddba991585791769dc9ede9.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..-4-misNGBjmPqXYelNn1vg.kbo3PpeHsU1h8Zdb5dScwsmFV9XvcJFtKLiY2OemgZDQmlFCRUhPOa-eDGrfjljCmrtCQG1Mby9DWu_jZkVkjiOagiCq0idnIvO_1eBfst17SdNPSH8fIVxRzEzVCOdII0fJnc3nAw1YGfXRE2kw1Yf6Gzv3M_A-g2XQQ3FgRsdrq_kXutojaqLGy_hHLtigRwx7apux9LeRneObnQNfQJP-qnmBjSIf7TTSuz4fc-Sxv0fdDE8dj_fzD2cRuzkU_nFSOBwKLpFP8L46CmlKSUYVZhnBKdn-kBE2WPt7udYDtxbeha8-An95yf_5tDHmc5KvmKKXPFiw5uStNP7G3DYnvSCjTfkJdWn4rfh3bJf9xYwkys6mk7jqfUHGE65jD3jofO-HZE5rS8-vt2JoTBWmAKlfcTDZDMXTFJFzpOKxJS0Nqdd45oQGZheXFwQF2U3QqcyeGPh9j_AJTfO5b8GVMt1XvFZrUvOrcOavThUjbyt4u51unhunnkl98QB0AlAvSDxxNBoo-sR3L-HA5Inmi6AUh5gmA8cJ-nLh5UW_4xNX7vsjM5K8GcvbZg9AjCNgJIlPcsXbaV4g4wMZDspKBF56EV50kA8kotou1xiuZUIFYhp4hkLpVFCzJgmq_0Q4bmawl6vT7w5_J5h-5g.ImYVWpK14W4RM06y4veA1g/5d35e9a318e04684020a42c3a2ca19fa.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..g-269sahKJQxSAdh-rDaAg._3dQWPmELvxkO-AvD1YIeqWdSpN7iU57tFKuH8xmA_4VbogyyLfvKGY-ttAf1gBIDyJWtBtXrKY7QygCPT2OBNU1e8QlYkLP21Ap4-VtDGGV_O8JoU6CEgEBc9-0i9KcpTCL2GRaps-7QB4XLLrVSIL8UtljUuOhQu0jTLape74FDUpYIwi_IFOtM20dlez8df_eIb9Jr7eCCdZk5wEKHfkxVCD4S8vjCSrh1m093xoGM1WAXFrnoRqRqewSZOsKJV4s8QJ6RQgajTWmS8CdXaEi7s_7-b0aMG0plO0580qsYpqMwB3EByrFlw7EnoXH-wqw3JQtgFsOtFpTx60185Rht60A-mVeR1_DKqkOPUJGEI3uzeVk4NOohNgKEsWMUf3m1TRzbmfieQQCZxUds7r69eGmgKYkIQJwbMtOhzZtlfAVYtAdM2ZxnRI-YHyvVD2ltK61SrlDAB4BoyWOx0kwJU1_2FTapH7AzZqUaRf398mGMhz077ja62qpJgMyMZOgI6qnoEooENE1a5_9YsrlOHgNSiEei4cTwVuQDTAujCes3Lc08aCY1enKXhlbxjdc7-D4UNC-A-tGYYNe8ZgfMmvvsYL7WzNNfaUCHozYef59AuBDxmr4VuqKElq01qO5iTA3qbhr5SMiWKwIvA.IIPG5ylIWNJLJeUlqkYIDw/2665dd5db932525f171f931c16112c65.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..vSUQ_ZmIseQD85ghmF0YHQ.ct1lcydWJfJyxtgngaR-CKZR1FJkugKEYOzAmOUHMWn_mY1MXlffhCC0RPenDcW6VKXhOYQdhFMia9DZa8JFgmN3gBJuJknS692oEOlP5qO1lYUqPKpnMGSiTUrTLdmvw02eyoW9TKs6Ie-50jduCGO524SlzZqLJb_YnYuFqyFeUhypgLR0ruRz4PA-be6HvgwPlfYIjrwHFVIFgkbnsrn_npFbZGiYUTVN3x48DhfpQTdUXX1KwvwEf4kTZOA5tCEy7DyYHORRF5Ly8kp4JxJ0KIn8g9LPIh2UfMLGJ21L_9DVG2r5TuRemukB16D8BgJ2olIGDiHk0g1bRRFGCmGU94ZkNkS8CH5nf_kQn3nxw12IrX7pDS4xNvse021wzjvyU3Oz9lDQWIKOrM_IhY44Qq_masxVfNeDFMdMv6qMiutfVVfKGmMrz5W2Tg_LlW114M8o2hPKIOy0_CL17GWHmreYt3-zSgo-c95p-y2tzEiMuoBS0_BaCufErClOdtaFVa2P9mK8_jpRIpnEgIgAXYpq3HDsgVbiWhApaGNwmd2E1A19Q0gE3dtiUC0V-2zMqAkuYrw9BQiGapUus3A2Lg5rftmFE0cfgrBpmOixcqwwKzDo0QkKjw5TkZqwJCYc3gKHqzxHZKes0dkWZg.3rStWhdYlqQNy7ncOOf5RA/0d7666fdc3372c68a080c612081e2b45.png',
                                  'https://www.kaggleusercontent.com/kf/44273081/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..PpAMYPWiHo7MhrmWzWmcSw.XessCLY0xikZmBGsWK-qwZHBUSNJjwr4vbZG9geWt0eiRrjOvxziy4cowx34oODiITPlhXW-dqY12-nMAWm0k5UVJ437IbSP4RKx1YOyfYfdxmMNUzwR4OcoaetuNDuv0qOgC2xsqGtbDmU070I2Sp_rFCVx79Ek-BR_3awp3hwqt56zEWqQvNVKZO-tbii54hEO3Em2evcFhxuXQHaVLoUgf-9TCBMeh0GzsN-JDs5X7x3IPRxqSUYcgcjiFXfcPbvRQUZ7TWyarxU5_vWyFRvDfFiOFaBhrBpNTvTiZPhJmt4QbTsCAHfCnJjpzs3FTRAunTfjgusOKI8HvCC2LnKBrDpB7uxPrtrDv7SRnsFaxn-IYrZhk2liTwVo6_4isAEZb_-QY6-Sm-GwH0XXPLqJ8mSabvj12hYVHvOojPjRhGUWRmQcXjlS1uPjLLNMa-PNPtqcglsnDO_b-cqd-_4wigGzuz_lh59w95sR7et4Uv0w7hgONBiv8F9H-xeMrBsUuV0ksblCTHe8Ub1yjIyvp9qZoaSvMVwEvRQzcz-2XTt3J6tuBoFHfhb_DH56VEfPKm-oEXJqSTZhMAr-RsHtYjc0qU-ePKmZCzO1GLOv3fljHMRyAqYnLN0SuZWA3lonL68IBG9ZxXBpPU9LeQ.yAfhVpwg2OFm8EqAfpDymw/fd1deb6f87696b6372ea9c9b109b4ccf.png']}

In [35]:
from PIL import Image as PILImage
def getSampleImageFromUrl(provider='radboud'):
    """Given a 'provider' this function will return an sample/random image
    from the notebook https://www.kaggle.com/jackbyte/sample-gleason-biopsy-pictures."""
    url = random.sample(list(provider_urls[provider]),1)[0]
    return PILImage.open(requests.get(url, stream=True).raw)


In [36]:
def calculateGridCoords(w, h, 
                        max_box_amount=100, 
                        init_pixels=256,
                        max_overlap=0.5):
    """Given widht 'w' and height 'h' this function will return a list
    of the x,y coords of (square) boxes and the pixel size of those boxes.
    
    The size of the boxes will be 'init_pixels' if the amount of the boxes is
    max_box_amount. If not, the size will be doubled until the amount is less
    than max_box_amount.
    x and ys of thiswithin the
    limits of 'w' and 'h'"""
    pixels = init_pixels
    coords = [i for i in range(max_box_amount)]
    too_many_boxes = True
    while too_many_boxes:
        
        # find x and y positions of box
        xes = [i*pixels for i in range(w//pixels)] 
        ys = [i*pixels for i in range(h//pixels)]
        
        # Add additional boxes if there is space left
        # and the additional box doesn't overlap more
        # than 'max_overlap'
        if (w%pixels!=0)&((w%pixels)/pixels>max_overlap):
            xes = xes + [w-pixels]
        if (h%pixels!=0)&((h%pixels)/pixels>max_overlap):
            ys = ys + [h-pixels]
        
        # list of all coordinates that have at least pixel_score pixels that are on the mask
        coords = [(x,y) for (x,y) in list(itertools.product(xes, ys)) ]
        
        
        if len(coords)>max_box_amount:
            pixels = pixels*2
        else:
            too_many_boxes = False
    
    return pixels, coords

In [37]:
def maskoff(img, coords, box_size, thr1=200, thr2=200, 
            krnlr=3,krnlc=3, target_dim=(8192, 26368), 
            pixel_score=0.25, verbose=False):
    """Given 'img', 'coords', and 'box_size' this function will  to return 
    - a resized version of the image
    - a matching image mask for the resized image
    - a image mask for the original image
    - a list of the x,y coords of (square 'box_size(d)') boxes 
      that overlap with the foreground of the original sized mask"""
    
    resized = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    
    edges = cv2.Canny(resized, thr1,thr2)
    kernel = np.ones((krnlr,krnlc), np.uint8)
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=3)
    erosion = cv2.morphologyEx(closing, cv2.MORPH_ERODE, kernel, iterations=1)

    # When using Grabcut the mask image should be:
    #    0 - sure background
    #    1 - sure foreground
    #    2 - unknown

    mask = np.zeros(resized.shape[:2], np.uint8)
    mask[:] = 2
    mask[erosion == 255] = 1

    bgdmodel = np.zeros((1, 65), np.float64)
    fgdmodel = np.zeros((1, 65), np.float64)

    out_mask = mask.copy()
    start = time.time()
    if verbose: print('[maskoff] resized.shape', resized.shape)
    out_mask, _, _ = cv2.grabCut(resized,out_mask,None,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_MASK)
    end = time.time()
    if verbose: print('[maskoff] grabcut took', end-start)
    out_mask = np.where((out_mask==2)|(out_mask==0),0,1).astype('uint8')
    
    out_mask_small = out_mask.copy()
    
    out_mask = cv2.resize(out_mask, target_dim, interpolation = cv2.INTER_CUBIC)
    
    # list of all coordinates that have at least pixel_score pixels that are on the mask
    out_coords = [(x,y) for (x,y) in coords if out_mask[y:y+box_size,x:x+box_size].sum()/(box_size*box_size)>pixel_score]
    
    return resized, out_mask_small, out_mask, out_coords

In [38]:


output_notebook()

TOOLTIPS = [("Classification", "@pred"),
            ("Gleason_3 confidence", "@Gleason_3"),
            ("Gleason_4 confidence", "@Gleason_4"),
            ("Gleason_5 confidence", "@Gleason_5"),
            ("benign_epithelium confidence", "@benign_epithelium")]

COLORMAP = { 'Gleason_3': 'yellow',
             'Gleason_4': 'salmon',
             'Gleason_5': 'red',
             'benign_epithelium': 'forestgreen'}

def bokehplotImageWithPatterns(image, confidence=0.7,  filename=None, verbose=False):
    """Given 'imgage' this function will return a bokeh figure containing
    - the input image
    - rectangle boxes that have been classified a Gleason pattern"""
    
    # resize the image 
    while sum(image.size)>10000:
        image.thumbnail(size=[v//2 for v in image.size])
    start = time.time()

    if verbose: print(image.size)

    # transform to cv image (maskoff uses cv2.grabCut) and flip (https://github.com/bokeh/bokeh/issues/1666)
    img_cv = np.array(image.convert('RGB'))[::-1]
    img = img_cv[:, :, ::-1].copy()

    w, h = image.size
    size, coords = calculateGridCoords(w, h, init_pixels=8)
    if verbose: print('Box size', size)
    #_, _, _, coords = maskoff(img, coords=crds, box_size=size, target_dim=image.size, pixel_score=0.05)
   
    p = figure(plot_width=800, plot_height=800, 
               x_range=(0,max(w,h)), y_range=(0,max(w,h)),
               tooltips=TOOLTIPS,
               title="Deteteced Gleason Patterns")
    
    im = image.convert("RGBA")
    # convert to numpy and flip (https://github.com/bokeh/bokeh/issues/1666)
    imarray = np.array(im)[::-1]
    p.image_rgba(image=[imarray], x=0,y=0, dw=w, dh=h)

    data = []
    # Classify the boxes
    for j, coord in enumerate(coords):
        x,y = coord
        patch = image.crop((x, y, x+size, y+size))
        c, l, preds = learn.predict(Image(T.ToTensor()(patch.convert('RGB'))))
        pname = learn.data.classes[l]
        tmp = dict(zip(learn.data.classes, ["{0:.2%}".format(p) for p in preds]))
        tmp['pred'] = pname
        tmp['x'] = x+size//2
        tmp['y'] = y+size//2
        tmp['line_clr'] = 'grey' if preds[l]<confidence else COLORMAP[pname]
        data.append(tmp)
        del patch
    end = time.time()
    df = pd.DataFrame(data)
    source = ColumnDataSource(data=df) 

    p.rect(x='x',y='y', width=size, height=size, line_color='line_clr', fill_color=None, source=source)
    
    if verbose: print('Took', end-start, 'to process image found', len(coords), 'patches')
    
    return p

Loading BokehJS ...

In [39]:


# defining widgets
karolinska_button = Button(description="Karolinska Biopsy", layout=Layout(height='auto', width='auto'))
radboud_button  = Button(description="Radboud Biopsy", layout=Layout(height='auto', width='auto'))
btn_upload = FileUpload(description="Upload Biopsy", multiple=False, layout=Layout(height='auto', width='auto'))
header_button = Button(description="Gleason Pattern Spotter", disabled=True, layout=Layout(height='auto', width='auto'))
footer_button = Button(description="Additional Information: Not Available", disabled=True, layout=Layout(height='auto', width='auto'))
output = Output(clear_output=True)

# defining event functions
def displayWaitMessage():
    footer_button.description = 'Processing - Please Wait'
    footer_button.style.button_color = 'yellow'

def displayReadyness():
    footer_button.description = 'Ready For User Input'
    footer_button.style.button_color = 'lightgreen'

def processSampleImage(provider='karolinska'):
    output.clear_output()
    with output:
        displayWaitMessage()
        img = getSampleImageFromUrl(provider)
        p = bokehplotImageWithPatterns(img)
        show(p)
        displayReadyness()
    
def on_karolinska_button_clicked(b):
    processSampleImage(provider='karolinska')

def on_radboud_button_clicked(b):
    processSampleImage(provider='radboud')
        

def on_data_change(change):
    displayWaitMessage()
    img = PILImage.open(io.BytesIO(btn_upload.data[-1]))
    with output:
        p = bokehplotImageWithPatterns(img)
        show(p)
    btn_upload._counter = 0
    displayReadyness()

# adding events
karolinska_button.on_click(on_karolinska_button_clicked)
radboud_button.on_click(on_radboud_button_clicked)
btn_upload.observe(on_data_change, names=['data'])

# Layout and Style 
applayout = AppLayout(header=header_button,
                      left_sidebar=karolinska_button,
                      right_sidebar=radboud_button,
                      footer=VBox([btn_upload,footer_button], layout=Layout(height='auto', width='auto')))
karolinska_button.style.button_color = 'lightsalmon'
radboud_button.style.button_color = 'aliceblue'

display(VBox([applayout,
              output]))